# 레스토랑 리뷰 감성 분류

In [1]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

## 데이터 벡터화 클래스

### Dataset

In [2]:
class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        '''
        :param review_df(pd.DataFrame): 데이터셋
        :param vectorizer: ReviewVectorizer 객체
        '''
        
        self.review_df = review_df
        self._vectorizer = vectorizer
        
        self.train_df = self.review_df[self.review_df.split == 'train']
        self.train_size = len(self.train_df)
        
        self.val_df = self.review_df[self.review_df.split == 'val']
        self.val_size = len(self.val_df)
        
        self.test_df = self.review_df[self.review_df.split == 'test']
        self.test_size = len(self.test_df)
        
        # 뭔지 주석 달아 놓을 것
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.val_size),
                             'test': (self.test_df, self.test_size)}
        
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        '''
        데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듦
        :param review_csv(str): 데이터셋의 위치
        :return: 
            ReviewDataset의 인스턴스
        '''
        
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split == 'train']
        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))
    
    def get_vectorizer(self):
        return self._vectorizer
    
    def set_split(self, split='train'):
        '''
        데이터프레임에 있는 열을 사용해 분할 세트를 선택
        :param split(str): 'train', 'val', 'test'중 하나
        '''
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        '''
        파이토치의 데이터셋 주요 진입 메서드
        :param index(int): 데이터 포인트의 인덱스
        :return: 
            데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
        '''
        row = self._target_df.iloc[index]
        
        review_vector = self._vectorizer.vectorize(row.review)
        rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)
        
        return {'x_data': review_vector,
                'y_target': rating_index}
    
    def get_num_batches(self, batch_size):
        '''
        배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다
        :param batch_size(int): 
        :return: 
            배치 개수
        '''
        return len(self) // batch_size

### Vocabulary

In [3]:
class Vocabulary(object):
    '''매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스'''
    
    def __init__(self, token_to_idx=None, add_unk=True, unk_token='<UNK>'):
        '''
        
        :param token_to_idx(dict): 기존 토큰-인덱스 매핑 딕셔너리
        :param add_unk(bool): UNK 토큰을 추가할지 지정하는 플래그
        :param unk_token(str): Vocabulary에 추가할 UNK 토큰
        '''
        
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
            
    def add_token(self, token):
        '''
        토큰을 기반으로 매핑 딕셔너리를 업데이트
        :param token(str): Vocabulary에 추가할 토큰
        :return: 
            index(int): 토큰에 상응하는 정수
        '''
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        '''
        토큰 리스트를 Vocabulary에 추가
        :param tokens(list): 문자열 토큰 리스트
        :return: 
            indices(list): 토큰 리스트에 상응되는 인덱스 리스트
        '''
        return [self.add_token(token) for token in tokens]
        
    def lookup_token(self, token):
        '''
        토큰에 대응하는 인덱스를 추출
        토큰이 없다면 UNK토큰을 반환
        :param token(str): 찾을 토큰
        :return: 
            index(int): 토큰에 해당하는 인덱스
        '''
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
        
    def lookup_index(self, index):
        '''
        인덱스에 해당하는 토큰을 반환
        :param index(int): 찾을 인덱스
        :return: 
            token(str): 인덱스에 해당하는 토큰
        :except
            KeyError: 인덱스가 없을 때 발생
        '''
        if index not in self._idx_to_token:
            raise KeyError('Vocabulary에 인덱스(%d)가 없습니다.' % index)
        return self._idx_to_token[index]
    
    def __str__(self):
        return '<Vocabulary(size=%d)>' % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

### Vectorizer

In [4]:
class ReviewVectorizer(object):
    '''어휘 사전을 생성하고 관리'''
    def __init__(self, review_vocab, rating_vocab):
        '''
        :param review_vocab(Vocabulary): 단어를 정수에 매핑하는 Vocabulary
        :param rating_vocab(Vocabulary): 클래스 레이블을 정수에 매핑하는 Vocabulary
        '''
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab
    
    def vectorize(self, review):
        '''
        리뷰에 대한 원-핫 벡터를 만듦
        :param review(str): 리뷰
        :return: 
            one-hot(np.ndarray): 원-핫 벡터
        '''
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        
        for token in review.split(' '):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1
        
        return one_hot
    
    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        '''
        데이터셋 데이터프레임에서 Vectorizer 객체를 만듦
        :param review_df(pd.DataFrame): 리뷰 데이터셋
        :param cutoff(int): 빈도 기반 필터링 설정값
        :return: 
            ReviewVectorizer 객체
        '''
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)
        
        # 점수를 추가
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)

        # count > cutoff인 단어를 추가
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
               
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)

        return cls(review_vocab, rating_vocab)

### DataLoader

In [5]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'):
    '''
    파이토치 DataLoader를 감싸고 있는 제너레이터 함수
    각 텐서를 지정된 장치로 이동함
    '''
    dataloader = DataLoader(dataset=dataset, 
                            batch_size=batch_size, 
                            shuffle=shuffle, 
                            drop_last=drop_last)
    
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

### ReviewClassifier 모델

In [6]:
class ReviewClassifier(nn.Module):
    '''간단한 퍼셉트론 분류기'''
    def __init__(self, num_features):
        '''
        :param num_features(int): 입력 특성 벡터의 크기
        '''
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
        
    def forward(self, x_in, apply_sigmoid=False):
        '''
        분류기의 정방향 계산
        :param x_in(torch.Tensor): 입력 데이터 텐서
            x_in.shape는 (batch, num_features)
        :param apply_sigmoid(bool): 시그모이드 활성화 함수를 위한 플래그
            크로스-엔트로피 손실을 사용하려면 False로 지정
        :return: 
            결과 텐서 tensor.shape는 (batch,)
        '''
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

## 훈련 과정

### 설정

In [7]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [8]:
args = Namespace(
    frequency_cutoff=25,
    model_state_file='model.pth',
    review_csv='data/reviews_with_splits_lite.csv',
    save_dir='model_storage/',
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True
)

if args.expand_filepaths_to_save_dir:
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)
    print('파일 경로: {}'.format(args.model_state_file))

if not torch.cuda.is_available():
    args.cuda = False

print('CUDA 사용 여부: {}'.format(args.cuda))

args.device = torch.device('cuda' if args.cuda else 'cpu')

set_seed_everywhere(args.seed, args.cuda)

handle_dirs(args.save_dir)

파일 경로: model_storage/model.pth
CUDA 사용 여부: False


### 헬퍼 함수

In [9]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    '''
    훈련 상태를 업데이트
    
    :param args: 메인 매개변수
    :param model: 훈련할 모델
    :param train_state: 훈련 상태를 담은 딕셔너리
    :return: 
        새로운 훈련 상태
    '''
    
    # 적어도 한 번 모델을 저장
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False
    
    # 성능이 향상되면 모델을 저장
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
        
        # 손실이 나빠지면
        if loss_t >= train_state['early_stopping_best_val']:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
        
        # 손실이 감소하면
        else:
            # 최상의 모델 저장
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                
            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0
        
        # 조기 종료 여부 확인
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria
        
    return train_state

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [10]:
print('데이터셋과 Vectorizer 만듦')
dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
vectorizer = dataset.get_vectorizer()

classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab))
classifier = classifier.to(args.device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, 
                                                 mode='min', 
                                                 factor=0.5, 
                                                 patience=1)
train_state = make_train_state(args)

데이터셋과 Vectorizer 만듦


### 훈련 반복

In [11]:
epoch_bar = tqdm(desc='training routine', 
                 total=args.num_epochs, 
                 position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train', 
                 total=dataset.get_num_batches(args.batch_size), 
                 position=1, 
                 leave=True)

dataset.set_split('val')
val_bar = tqdm(desc='split=val', 
               total=dataset.get_num_batches(args.batch_size), 
               position=1, 
               leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index
        
        # 훈련 세트에 대한 순회
        
        # 훈련 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            # 훈련은 5단계로 이루어짐
            
            # 1. 그레이디언트를 0으로 초기화
            optimizer.zero_grad()
            
            # 2. 출력을 계산
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            
            # 3. 손실을 계산
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            
            # 4. 손실을 사용해 그레이디언트를 계산
            loss.backward()
            
            # 5. 옵티마이저로 가중치를 업데이트
            optimizer.step()
            
            # 정확도 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # 진행바 업데이트
            train_bar.set_postfix(loss=running_loss,
                                  acc=running_acc,
                                  epoch=epoch_index)
            train_bar.update()
            
        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)
        
        # 검증 세트에 대한 순회
        
        # 검증 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            
            # 1. 출력을 계산
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            
            # 2. 손실을 계산
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            
            # 3. 정확도를 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # 진행바 업데이트
            val_bar.set_postfix(loss=running_loss,
                                  acc=running_acc,
                                  epoch=epoch_index)
            val_bar.update()
            
        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        
        train_state = update_train_state(args=args, 
                                         model=classifier, 
                                         train_state=train_state)
        scheduler.step(train_state['val_loss'][-1])
        
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
        if train_state['stop_early']:
            break
            
        # train_bar.n = 0
        # val_bar.n = 0
        # epoch_bar.update()
        
except KeyboardInterrupt:
    print("Exiting loop")

training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/306 [00:00<?, ?it/s]

split=val:   0%|          | 0/65 [00:00<?, ?it/s]

In [12]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산
classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력을 계산합니다
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    # 손실을 계산합니다
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산합니다
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [13]:
print("테스트 손실: {:.3f}".format(train_state['test_loss']))
print("테스트 정확도: {:.2f}".format(train_state['test_acc']))

테스트 손실: 0.222
테스트 정확도: 91.55


### 추론

In [14]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [19]:
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    '''
    리뷰 점수 예측하기
    :param review(str): 리뷰 텍스트
    :param classifier(ReviewClassifier): 훈련된 모델
    :param vectorizer(ReviewVectorizer): Vectorizer 객체
    :param decision_threshold(float): 클래스를 나눌 경계
    :return: 
    '''
    review = preprocess_text(review)
    
    vectorizer_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorizer_review.view(1, -1))
    
    probability_value = torch.sigmoid(result).item()
    index = 1
    if probability_value < decision_threshold:
        index = 0
    
    return vectorizer.rating_vocab.lookup_index(index)



In [20]:
test_review = "this is a pretty awesome book"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

this is a pretty awesome book -> positive


### 해석

In [21]:
classifier.fc1.weight.shape

torch.Size([1, 9323])

In [23]:
# 가중치 정렬
fc1_weights = classifier.fc1.weight.detach()[0]
_, indices = torch.sort(fc1_weights, dim=0, descending=True)
indices = indices.numpy().tolist()

# 긍정적인 상위 20개 단어
print("긍정 리뷰에 영향을 미치는 단어:")
print("--------------------------------------")
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))
    
print("====\n\n\n")

# 부정적인 상위 20개 단어
print("부정 리뷰에 영향을 미치는 단어:")
print("--------------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

긍정 리뷰에 영향을 미치는 단어:
--------------------------------------
delicious.
pleasantly
great!
fantastic.
amazing
excellent
great
fantastic
ngreat
perfect
awesome.
delicious
favorites
notch.
awesome!
excellent.
fantastic!
delicious!
vegas!
amazing.
====



부정 리뷰에 영향을 미치는 단어:
--------------------------------------
worst
terrible.
awful.
mediocre
horrible.
bland
horrible
rude
meh.
rude.
poorly
disappointing.
bland.
unfriendly
overpriced
mediocre.
terrible
desired.
slowest
tasteless
